# `OneVsRestClassifier` vs `MultiOutputClassifier`

In [54]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import log_loss, f1_score, accuracy_score

In [41]:
x, y = make_multilabel_classification(n_samples=500, n_features=20,
    n_classes=5, n_labels=2, allow_unlabeled=False,
    random_state=42)
x_tr, x_val, y_tr, y_val = train_test_split(x, y, test_size=0.2,
                                            random_state=42, stratify=y)

In [42]:
clf = LogisticRegression()
clf_ovr = OneVsRestClassifier(clf)
clf_moc = MultiOutputClassifier(clf)

In [43]:
clf_ovr.fit(x_tr, y_tr)
preds_ovr = clf_ovr.predict(x_val)
proba_ovr = clf_ovr.predict_proba(x_val)

In [44]:
clf_moc.fit(x_tr, y_tr)
preds_moc = clf_moc.predict(x_val)
proba_moc = clf_moc.predict_proba(x_val)

In [45]:
proba_ovr.shape

(100, 5)

In [46]:
len(proba_moc), proba_moc[0].shape

(5, (100, 2))

In [47]:
log_loss(y_val.ravel(), proba_ovr.ravel())

0.4074235265298689

In [48]:
proba_ovr[:5]

array([[0.00834483, 0.76214308, 0.9738294 , 0.31079416, 0.18031951],
       [0.43231617, 0.99863548, 0.93038194, 0.10891879, 0.00443261],
       [0.9999991 , 0.02551868, 0.47593717, 0.05788946, 0.02618498],
       [0.05052963, 0.9801258 , 0.76264921, 0.78499218, 0.01984307],
       [0.18210457, 0.15661448, 0.13703427, 0.99730301, 0.25770792]])

The multi-output predictions have the probability of the 0 label in the first column and for the 1 label in the second column.

In [49]:
proba_moc[0][:5]

array([[9.91655171e-01, 8.34482867e-03],
       [5.67683832e-01, 4.32316168e-01],
       [8.99624308e-07, 9.99999100e-01],
       [9.49470367e-01, 5.05296327e-02],
       [8.17895432e-01, 1.82104568e-01]])

However, when we ravel, we have the 5 class probabilities for the first sample, followed by the 5 class probabilities for the second, and so on.

In [50]:
proba_ovr.ravel()[:5]

array([0.00834483, 0.76214308, 0.9738294 , 0.31079416, 0.18031951])

Same thing, obviously, for the class labels.

In [51]:
print(y_val[:5])
print(y_val.ravel()[:5])

[[0 0 1 1 0]
 [1 1 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 0 1 0]]
[0 0 1 1 0]


According to [this page](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) this is the correct approach.

In [55]:
accuracy_score(y_val, preds_ovr)

0.32

While this is incorrect. **WHY?**

In [56]:
accuracy_score(y_val.ravel(), preds_ovr.ravel())

0.81

In [58]:
f1_score(y_val.ravel(), preds_ovr.ravel())

0.7912087912087912

In [59]:
accuracy_score(y_val, preds_moc)

0.32